# compiling cloud passes


In [12]:
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from glob import glob
import seaborn as sns
#import xesmf as xe
import pandas as pd
#import def_homebrew as hb ## homemade functions xox
from scipy.special import gamma
import netCDF4 as nc
from datetime import datetime, timedelta

import h5py ####
from PIL import Image
#from IPython.display import display #
#import cv2 # not working
import os


In [ ]:
## all info about flight data is in 
faam_loc = '/badc/faam/data/2022/' ## this contains all faam flights - more than just DCMEX
file_list = glob(faam_loc+'*/') # whole string of path + filenames in location
file_names = [os.path.basename(os.path.dirname(file_path)) for file_path in file_list] # extract just directory names

## list just DCMEX flight:
start_file = "c298"
end_file = "c316"

## Extract files between the start_file and end_file inclusive
#(should be noted flight c309 is in this list - but no corresponding data)
selected_files = [f for f in file_names if f >= start_file and f <= end_file]
## e.g. 'c307-jul-30'

In [ ]:
loc = faam_loc + 'c307-jul-30/'
csv = glob(loc +'*.csv') # get the csv file containing flight summary for that day
df = pd.read_csv(csv[0])

In [36]:
df = pd.read_csv('flight-sum_faam_20220730_r0_c307.csv') ## whole csv
df.set_index('Event', inplace=True) # set the event as index for df
run_df = df[df.index.str.contains('Run')] # df with only cloud passes info

run_df

,Start,Start Hdg / °,Start Hgt / kft,Start Lat / °,Start Long / °,Stop,Stop Hdg / °,Stop Hgt / kft,Stop Lat / °,Stop Long / °,Comment
Event,,,,,,,,,,,
Run 5.8,30/07/2022 19:20,88,18.012,33.9412,-107.258,30/07/2022 19:22,89.0,18.024,33.9547,-107.058,18 kft
Run 5.7,30/07/2022 19:15,266,20.992,33.9497,-107.109,30/07/2022 19:17,174.0,19.992,33.9007,-107.316,21 kft
Run 5.6,30/07/2022 19:10,62,20.620,33.9131,-107.288,30/07/2022 19:12,78.0,20.528,33.9503,-107.127,20.5 kft
Run 5.5,30/07/2022 19:05,258,19.396,33.9354,-107.082,30/07/2022 19:07,258.0,19.500,33.9079,-107.265,19.5 kft
Run 5.4,30/07/2022 19:00,34,19.276,33.8633,-107.275,30/07/2022 19:02,37.0,18.896,34.0010,-107.130,19 kft
Run 5.3,30/07/2022 18:54,239,18.564,33.9888,-107.051,30/07/2022 18:57,229.0,18.536,33.8697,-107.252,18.5 kft
Run 5.2,30/07/2022 18:50,51,18.032,33.8583,-107.244,30/07/2022 18:53,293.0,18.488,33.9988,-107.002,18 kft
Run 5.1,30/07/2022 18:42,236,13.700,34.0531,-106.964,30/07/2022 18:46,225.0,13.732,33.8861,-107.217,Cloud runs
Run 4.4,30/07/2022 18:17,337,8.716,33.6831,-107.167,30/07/2022 18:21,336.0,8.700,33.9657,-107.343,DC3 - DC4


In [39]:
specif_run = 'Run 2.2'

start_time =  run_df.loc[specif_run, 'Start'][-5:] # just give time begin cloud pass
stop_time = run_df.loc[specif_run, 'Stop'][-5:] # time of end of cloud pass